<a href="https://colab.research.google.com/github/oluwafemidiakhoa/Mindserach/blob/master/MAS_Multiagentsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install python-dotenv
!pip install openai
!pip install google-colab
!pip install networkx



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.0 MB/s eta 0:00:00


In [46]:
import os
import queue
import threading
from dotenv import load_dotenv
from openai import OpenAI
from google.colab import userdata
import networkx as nx


# Load environment variables from the .env file
load_dotenv()

# Initialize the OpenAI client
client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY")  # Fetch API key from environment variables or Colab user data
)
# Blackboard for Sharing Results
class Blackboard:
    def __init__(self):
        self.data = {}
        self.lock = threading.Lock()

    def write(self, key, value):
        with self.lock:
            self.data[key] = value

    def read(self, key):
        with self.lock:
            return self.data.get(key, None)

# Communication Channel for Agent Interaction
class CommunicationChannel:
    def __init__(self):
        self.queues = {}

    def register_agent(self, agent_id):
        self.queues[agent_id] = queue.Queue()

    def send(self, sender_id, recipient_id, content):
        if recipient_id in self.queues:
            self.queues[recipient_id].put({"sender": sender_id, "content": content})

    def receive(self, agent_id):
        if agent_id in self.queues and not self.queues[agent_id].empty():
            return self.queues[agent_id].get()
        return None


In [47]:
class Agent(threading.Thread):
    def __init__(self, agent_id=None):
        super().__init__()
        self.agent_id = agent_id or f"agent-{id(self)}"
        self.role = "Agent"
        self.memory = []
        self.communication_channel = None
        self.running = True

    def communicate(self, message, recipient_id=None):
        if self.communication_channel:
            self.communication_channel.send(self.agent_id, recipient_id, message)

    def receive_messages(self):
        return self.communication_channel.receive(self.agent_id) if self.communication_channel else None

    def query_gpt(self, prompt):
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="gpt-4o"
        )
        return response.choices[0].message.content

    def run(self):
        while self.running:
            message = self.receive_messages()
            if message:
                self.perform_task(message["content"])

    def perform_task(self, input_data):
        pass  # To be overridden by subclasses


In [48]:
class MissionPlanningAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = "Mission Planning Agent"

    def perform_task(self, mission_objectives):
        prompt = f"Create a detailed mission plan based on the following objectives:\n{mission_objectives}"
        mission_plan = self.query_gpt(prompt)
        blackboard.write("mission_plan", mission_plan)
        self.memory.append({"task": "mission_planning", "data": mission_plan})
        print(f"[{self.role}] Mission plan created: {mission_plan}")
        return mission_plan


In [49]:
class DataAnalysisAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = "Data Analysis Agent"

    def perform_task(self, data_description):
        prompt = f"Analyze the following dataset and provide insights:\n{data_description}"
        analysis = self.query_gpt(prompt)
        blackboard.write("data_analysis", analysis)
        self.memory.append({"task": "data_analysis", "data": analysis})
        print(f"[{self.role}] Analysis completed: {analysis}")
        return analysis


In [50]:
class AnomalyDetectionAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = "Anomaly Detection Agent"

    def perform_task(self, data_description):
        try:
            import numpy as np
            data = np.array(eval(data_description))
            mean, std = np.mean(data), np.std(data)
            anomalies = [x for x in data if abs(x - mean) > 2 * std]
        except Exception:
            anomalies = "Unable to process data offline."

        if isinstance(anomalies, str):
            prompt = f"Detect anomalies in the following data:\n{data_description}"
            anomalies = self.query_gpt(prompt)

        blackboard.write("anomaly_detection", anomalies)
        self.memory.append({"task": "anomaly_detection", "data": anomalies})
        print(f"[{self.role}] Anomalies detected: {anomalies}")
        return anomalies


In [51]:
import networkx as nx

class KnowledgeGraphAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = "Knowledge Graph Agent"
        self.graph = nx.Graph()

    def add_relation(self, entity1, entity2, relation):
        self.graph.add_edge(entity1, entity2, relation=relation)

    def query_relation(self, entity1, entity2):
        if self.graph.has_edge(entity1, entity2):
            return self.graph.get_edge_data(entity1, entity2)
        return None

    def perform_task(self, task_description):
        if "add relation" in task_description:
            entity1, entity2, relation = task_description.split(",")[1:]
            self.add_relation(entity1.strip(), entity2.strip(), relation.strip())
            response = f"Relation '{relation}' added between {entity1} and {entity2}."
        elif "query relation" in task_description:
            entity1, entity2 = task_description.split(",")[1:]
            response = self.query_relation(entity1.strip(), entity2.strip())
            response = response or f"No relation found between {entity1} and {entity2}."
        else:
            response = "Invalid task for Knowledge Graph Agent."

        blackboard.write("knowledge_graph_response", response)
        print(f"[{self.role}] {response}")
        return response


In [53]:
class EthicsAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = "Ethics Agent"

    def evaluate_ethics(self, decision):
        if "risk" in decision.lower():
            return "Ethical concern: High-risk decision. Consider alternatives."
        return "Decision is ethically sound."

    def perform_task(self, decision_description):
        ethical_assessment = self.evaluate_ethics(decision_description)
        blackboard.write("ethics_evaluation", ethical_assessment)
        print(f"[{self.role}] Ethics evaluation: {ethical_assessment}")
        return ethical_assessment


In [56]:
import random

class SimulationAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = "Simulation Agent"

    def simulate_scenario(self, scenario_description):
        outcomes = [
            "Mission success with 95% confidence",
            "Resource depletion detected in 30% of cases",
            "Critical system failure under extreme conditions",
        ]
        return random.choice(outcomes)

    def perform_task(self, scenario_description):
        simulation_result = self.simulate_scenario(scenario_description)
        blackboard.write("simulation_result", simulation_result)
        print(f"[{self.role}] Simulation result: {simulation_result}")
        return simulation_result


In [57]:
def enhanced_workflow(mission_objectives, agents):
    for agent in agents:
        communication_channel.register_agent(agent.agent_id)
        agent.communication_channel = communication_channel

    planning_agent = next(agent for agent in agents if agent.role == "Mission Planning Agent")
    print(f"\n[{planning_agent.role}] Starting mission planning...")
    mission_plan = planning_agent.perform_task(mission_objectives)

    data_agent = next(agent for agent in agents if agent.role == "Data Analysis Agent")
    print(f"\n[{data_agent.role}] Starting data analysis...")
    data_agent.perform_task("Simulated lunar data: [1.2, 2.3, 1.8]")

    simulation_agent = next(agent for agent in agents if agent.role == "Simulation Agent")
    print(f"\n[{simulation_agent.role}] Running simulation...")
    simulation_agent.perform_task("Evaluate lunar landing risks.")

    ethics_agent = next(agent for agent in agents if agent.role == "Ethics Agent")
    print(f"\n[{ethics_agent.role}] Evaluating mission ethics...")
    ethics_agent.perform_task(mission_plan)

    print("\nFinal Results:")
    for key, value in blackboard.data.items():
        print(f"{key.capitalize()}: {value}")


In [58]:
# Initialize Blackboard and Communication Channel
blackboard = Blackboard()
communication_channel = CommunicationChannel()

# Instantiate Enhanced Agents
agents = [
    MissionPlanningAgent(),
    DataAnalysisAgent(),
    AnomalyDetectionAgent(),
    SimulationAgent(),
    KnowledgeGraphAgent(),
    EthicsAgent(),
]

# Define Mission Objectives
mission_objectives = "Conduct a study of the lunar South Pole for geological anomalies and water ice detection."

# Execute Enhanced Workflow
enhanced_workflow(mission_objectives, agents)



[Mission Planning Agent] Starting mission planning...
[Mission Planning Agent] Mission plan created: Creating a detailed mission plan for a study of the lunar South Pole with a focus on geological anomalies and water ice detection involves several key phases: mission preparation, launch, transit, lunar surface operations, data collection, and return. Below is a detailed mission outline:

### Mission Plan

---

#### Phase 1: Mission Preparation

**1. Objectives Definition**
   - Clearly define scientific goals: geological anomaly mapping and water ice detection.
   - Establish success criteria for each objective.

**2. Team Assembly**
   - Assemble a multidisciplinary team, including geologists, astrophysicists, engineers, and mission planners.
   - Designate a mission leader and specific roles for team members.

**3. Spacecraft Design and Instruments**
   - Select a suitable lander or rover equipped with necessary instruments:
     - Radar systems for subsurface scanning.
     - Spect